In [28]:
import numpy as np
import scipy as sp
import pandas as pd
import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt
import time
import random
from scipy import linalg
from scipy.stats import norm
from scipy.stats import t
from scipy.stats import truncnorm
from scipy.stats import truncexpon
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from keras import backend as K
from keras.models import Sequential
from keras.layers import Flatten,Conv2D,Dropout,Input,Dense,MaxPooling2D
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.utils.vis_utils import plot_model
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from keras.wrappers.scikit_learn import KerasClassifier
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示中文标签
from ERANataf import ERANataf
from ERADist import ERADist
# from SuS import SuS
from SuS_0 import SuS
plt.close('all')

In [4]:
def latin_hypercube(n, d):
    """
    生成拉丁超立方抽样点集合

    Args:
        n: 抽样点数量
        d: 维度

    Returns:
        返回一个形状为 (n, d) 的 NumPy 数组，其中包含 n 个 d 维均匀分布的抽样点
    """
    # 生成均匀分布的样本点
    x = np.zeros((n, d))
    # for i in range(d):
    #     x[:, i] = np.random.uniform(size=n)
    # 对每一列进行排列
    for i in range(d):
        order = np.random.permutation(range(n))
        x[:, i] = (order + np.random.uniform(size=n)) / n
    return x


In [5]:
def f(u):
    y = u[:, 0] + u[:, 1] + 6
    # print('111')
    return(y)

In [21]:
time_start = time.time()

P_res_1 = np.zeros(100)
for i in range(100):
    N = 100000000
    d = 2
    random_seed = latin_hypercube(N, d)
    # x1 = np.random.normal(0, 1, size = (N, 1))
    # x2 = np.random.normal(0, 1, size = (N, 1))
    # x3 = np.random.normal(0, 1, size = (N, 1))
    x1 = truncnorm.ppf(random_seed[:, 0], a = -5, b = 5, loc = 0, scale = 1)
    x2 = truncnorm.ppf(random_seed[:, 1], a = -5, b = 5, loc = 0, scale = 1)

    y = x1 + x2 + 6

    for j in range(N):
        if y[j] < 0:
            P_res_1[i] = P_res_1[i] + 1/N
    print(i)
P_res_1
time_end = time.time()
time_CPU_1 = time_end - time_start

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


array([1.103e-05, 1.074e-05, 1.105e-05, 1.117e-05, 1.046e-05, 1.055e-05,
       1.095e-05, 1.084e-05, 1.086e-05, 1.128e-05, 1.113e-05, 1.107e-05,
       1.101e-05, 1.045e-05, 1.117e-05, 1.120e-05, 1.151e-05, 1.066e-05,
       1.083e-05, 1.097e-05, 1.054e-05, 1.120e-05, 1.161e-05, 1.101e-05,
       1.094e-05, 1.048e-05, 1.062e-05, 1.122e-05, 1.105e-05, 1.060e-05,
       1.074e-05, 1.133e-05, 1.053e-05, 1.095e-05, 1.115e-05, 1.091e-05,
       1.094e-05, 1.054e-05, 1.090e-05, 1.107e-05, 1.048e-05, 1.073e-05,
       1.066e-05, 1.122e-05, 1.130e-05, 1.055e-05, 1.096e-05, 1.121e-05,
       1.039e-05, 1.056e-05, 1.113e-05, 1.103e-05, 1.120e-05, 1.129e-05,
       1.082e-05, 1.098e-05, 1.122e-05, 1.077e-05, 1.094e-05, 1.161e-05,
       1.043e-05, 1.069e-05, 1.088e-05, 1.101e-05, 1.091e-05, 1.089e-05,
       1.077e-05, 1.120e-05, 1.061e-05, 1.187e-05, 1.071e-05, 1.087e-05,
       1.094e-05, 1.096e-05, 1.050e-05, 1.110e-05, 1.069e-05, 1.113e-05,
       1.071e-05, 1.089e-05, 1.111e-05, 1.025e-05, 

In [22]:
np.mean(P_res_1), np.var(P_res_1), min(P_res_1), max(P_res_1)

(1.0949699999999906e-05,
 9.713690999999814e-14,
 1.0249999999999912e-05,
 1.1869999999999897e-05)

In [26]:
"""
---------------------------------------------------------------------------
Subset Simulation: 多项式案例: y = x1 + x2 + 6
---------------------------------------------------------------------------
"""

time_start = time.time()
# %% definition of the random variables
d      = 2          # number of dimensions
pi_pdf = list()
for i in range(d):
    pi_pdf.append(ERADist('truncatednormal','PAR',[0, 1, -5, 5])) # n independent rv

# correlation matrix
R = np.eye(d)   # independent case

# object with distribution information
pi_pdf = ERANataf(pi_pdf, R)    # if you want to include dependence

# %% limit state function
g = lambda u: f(u)

# g = ANN(u, model, x, y)


# %% subset simulation
# P_res_1 = np.zeros(100)
P_res_2 = np.zeros(100)
for k in range(100):
    N  = 1000      # Total number of samples for each level
    # N  = 100000        # Total number of samples for each level
    p0 = 0.1         # Probability of each subset, chosen adaptively

    # Implementation of sensitivity analysis: 1 - perform, 0 - not perform
    sensitivity_analysis = 0

    # Samples return: 0 - none, 1 - final sample, 2 - all samples
    samples_return = 1

    print('\n\nSUBSET SIMULATION: ')
    [Pf_SuS, delta_SuS, b, Pf, b_sus, pf_sus, samplesU, samplesX, S_F1] = SuS(N, p0, g, pi_pdf, sensitivity_analysis, samples_return)

    # Reference values
    # The reference values for the first order indices
    S_F1_ref   = [0.0000117, 0.0000117]  # 这里是蒙卡结果

    # Print reference values for the first order indices
    print("\n***Reference first order Sobol' indices: ***\n", S_F1_ref)

    # # exact solution
    # pf_ex    = sp.stats.norm.cdf(-beta)
    # Pf_exact = lambda gg: sp.stats.norm.cdf(gg,beta,1)
    # gg       = np.linspace(0,7,140)

    # show p_f results
    # print('\n***Exact Pf: ', pf_ex, ' ***')
    print('***SuS Pf: ', Pf_SuS, ' ***\n')

    X = np.array(samplesX).ravel().reshape(-1,2)

    z_m = f(X)

    output = np.hstack((X, z_m.reshape(-1,1)))
    res = output[np.argsort(-output[:, 2])]        #对输出值降序排列（带负号就是降序）
    df = pd.DataFrame(res).drop_duplicates()       #去除重复行
    z_m = df.values

    x_FC = z_m[:, 0:2]
    y_FC = z_m[:, 2]

    P_conditional_m_true = truncnorm.cdf(-x_FC[:, 1]-6, a = -5, b = 5, loc = 0, scale = 1)
    P_conditional_x_true = truncnorm.cdf(-x_FC[:, 0]-6, a = -5, b = 5, loc = 0, scale = 1)

    # P_res_1[k] = sum(P_conditional_x_true * P_conditional_m_true)/len(P_conditional_x_true)

    P_f = 0
    w = 0.01
    # m = 30
    m = len(x_FC)
    # for i in range(len(x_FC)):
    for i in range(m):
        f_x = truncnorm.pdf(x_FC[i, 0], a = -5, b = 5, loc = 0, scale = 1) * truncnorm.pdf(x_FC[i, 1], a = -5, b = 5, loc = 0, scale = 1) 
        # f_x = 1
        g_x = 0
        for j in range(m):
            if x_FC[i, 0] + x_FC[j, 1] + 6 < 0:
                g_x = g_x + f_x /(P_conditional_m_true[j] * P_conditional_x_true[i])
                # print(m_true)
            # g_x = g_x + f_x /1.162e-5
        g_x = g_x / m
        if g_x != 0:
            P_f = P_f + f_x / g_x
        print(i)
    # P_f = P_f / len(x_FC)
    P_f = P_f / m
    P_res_2[k] = P_f
    print('k = %d'%k)
time_end = time.time()
time_CPU_2 = time_end - time_start



SUBSET SIMULATION: 
Evaluating performance function:	OK!

-Threshold intermediate level  0  =  4.152753841781559
	*aCS lambda = 0.7464937472681182 	*aCS sigma = 0.7464937472681182 	*aCS accrate = 0.4802469135802469

-Threshold intermediate level  1  =  2.671890467308439
	*aCS lambda = 0.513303442830521 	*aCS sigma = 0.513303442830521 	*aCS accrate = 0.37530864197530867

-Threshold intermediate level  2  =  1.6729971919417026
	*aCS lambda = 0.42505817893344766 	*aCS sigma = 0.42505817893344766 	*aCS accrate = 0.4148148148148148

-Threshold intermediate level  3  =  0.8780684192927714
	*aCS lambda = 0.3810241552297271 	*aCS sigma = 0.3810241552297271 	*aCS accrate = 0.41851851851851857

-Threshold intermediate level  4  =  0.09530589428741304
	*aCS lambda = 0.3583248785430099 	*aCS sigma = 0.3583248785430099 	*aCS accrate = 0.43580246913580245

-Threshold intermediate level  5  =  0.0
	*aCS lambda = 0.33602978338315825 	*aCS sigma = 0.33602978338315825 	*aCS accrate = 0.333333333333333

In [15]:
P_res_1, P_res_2

(array([1.0e-05, 1.6e-05, 1.5e-05, 9.0e-06, 7.0e-06, 1.6e-05, 1.1e-05,
        1.8e-05, 1.1e-05, 8.0e-06, 8.0e-06, 3.0e-06, 1.3e-05, 1.3e-05,
        1.8e-05, 8.0e-06, 1.0e-05, 9.0e-06, 1.1e-05, 1.2e-05, 1.2e-05,
        1.4e-05, 7.0e-06, 1.5e-05, 8.0e-06, 9.0e-06, 9.0e-06, 8.0e-06,
        1.0e-05, 1.6e-05, 7.0e-06, 1.1e-05, 1.5e-05, 8.0e-06, 1.1e-05,
        8.0e-06, 1.2e-05, 9.0e-06, 7.0e-06, 8.0e-06, 1.7e-05, 1.8e-05,
        9.0e-06, 1.5e-05, 7.0e-06, 7.0e-06, 8.0e-06, 1.4e-05, 1.5e-05,
        1.5e-05, 1.9e-05, 1.2e-05, 1.0e-05, 1.2e-05, 7.0e-06, 1.1e-05,
        9.0e-06, 8.0e-06, 1.2e-05, 1.0e-05, 7.0e-06, 1.0e-05, 1.3e-05,
        1.1e-05, 1.1e-05, 9.0e-06, 7.0e-06, 9.0e-06, 1.4e-05, 1.1e-05,
        1.1e-05, 1.1e-05, 1.0e-05, 1.3e-05, 1.6e-05, 1.2e-05, 1.2e-05,
        1.0e-05, 1.1e-05, 1.2e-05, 8.0e-06, 1.2e-05, 1.2e-05, 1.3e-05,
        1.5e-05, 1.1e-05, 1.4e-05, 1.0e-05, 1.4e-05, 8.0e-06, 1.1e-05,
        8.0e-06, 6.0e-06, 1.5e-05, 8.0e-06, 1.4e-05, 9.0e-06, 1.1e-05,
      

In [39]:
time_start = time.time()
P_res_3 = np.zeros(100)
for k in range(100):
    n = int(2e4)
    d = 2
    P_f = 0
    random_seed = latin_hypercube(n, d)

    x1_rvs = truncexpon.ppf(random_seed[:, 0], b = 10/2, loc = -5, scale = 2)
    x2_rvs = truncexpon.ppf(random_seed[:, 1], b = 10/2, loc = -5, scale = 2)
    x_rvs = np.hstack((x1_rvs.reshape(-1,1), x2_rvs.reshape(-1,1)))

    y_rvs = f(x_rvs)

    f_p = 0
    w1 = truncnorm.pdf(x1_rvs, a = -5, b = 5, loc = 0, scale = 1) / truncexpon.pdf(x1_rvs, b = 10/2, loc = -5, scale = 2)
    w2 = truncnorm.pdf(x2_rvs, a = -5, b = 5, loc = 0, scale = 1) / truncexpon.pdf(x2_rvs, b = 10/2, loc = -5, scale = 2)
    for i in range(n):
        if y_rvs[i] < 0:
            f_p = f_p + w1[i] * w2[i]
    P_res_3[k] = f_p / n
time_end = time.time()
time_CPU_3 = time_end - time_start

In [23]:
np.mean(P_res_1), np.var(P_res_1), min(P_res_1), max(P_res_1), time_CPU_1

(1.0949699999999906e-05,
 9.713690999999814e-14,
 1.0249999999999912e-05,
 1.1869999999999897e-05)

In [27]:
np.mean(P_res_2), np.var(P_res_2), min(P_res_2), max(P_res_2), time_CPU_2

(1.0975844725615068e-05,
 1.0658308997369184e-13,
 9.68511901007831e-06,
 1.1890703912636719e-05,
 169.03784847259521)

In [40]:
np.mean(P_res_3), np.var(P_res_3), min(P_res_3), max(P_res_3), time_CPU_3

(1.0950032954354945e-05,
 1.0122614897848177e-13,
 1.0259527564262702e-05,
 1.1619620682747681e-05,
 1.257657527923584)